In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, KFold
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [4]:
data = pd.read_csv('alldata-DNA.csv')

In [21]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.2, random_state= 181 )

In [22]:
X_train = train_set.drop(columns=['Y'], axis=1).values
y_train = train_set.drop(columns=['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16'], axis=1).values
X_test = test_set.drop(columns=['Y'], axis=1).values 
y_test = test_set.drop(columns=['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16'], axis=1).values

In [23]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

1、XGB

In [42]:
xgb_model = xgb.XGBRegressor(random_state = 42,
                                 learning_rate= 0.188,
                                 max_depth =  6,
                                 n_estimators = 42,
                                 reg_alpha= 0.001, # Add L1 regularization
                                 reg_lambda= 1)
    

model = xgb_model.fit(X_train,y_train)
testpre = xgb_model.predict(X_test)
trainpre = xgb_model.predict(X_train)
mse_in_test = mean_squared_error (y_test , testpre)
rmse_in_test = np.sqrt (mse_in_test)
R2_in_test = r2_score(y_test , testpre)
# 保存到Excel文档中

trainpre_df = pd.DataFrame(trainpre)
testpre_df = pd.DataFrame(testpre)
trainpre_df.to_excel("trainpre2.xlsx", index=False)
testpre_df.to_excel("testpre2.xlsx", index=False)
y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_test)
y_train_df.to_excel("y_train2.xlsx", index=False)
y_test_df.to_excel("y_test2.xlsx", index=False)
mse_in_train = mean_squared_error (y_train , trainpre)
rmse_in_train = np.sqrt (mse_in_train)
R2_in_train = r2_score(y_train , trainpre)

print ("xgb回归模型的RMSE-训练集：",  rmse_in_train)
print ("xgb回归模型的R2-训练集：", R2_in_train)
print ("xgb回归模型的RMSE-测试集：",  rmse_in_test)
print ("xgb回归模型的R2-测试集：", R2_in_test)
print ("测试集Y：",  y_test)
print ("测试集Y预测值：",  testpre)
print ("训练集Y：",  y_train)
print ("训练集Y预测值：",  trainpre)

xgb回归模型的RMSE-训练集： 0.041920650830681284
xgb回归模型的R2-训练集： 0.997829450584916
xgb回归模型的RMSE-测试集： 0.28943500546335404
xgb回归模型的R2-测试集： 0.8051143844629859
测试集Y： [[3.28642107]
 [2.95537286]
 [3.4211577 ]
 [3.40049936]
 [3.9169024 ]
 [3.6290245 ]
 [2.74673374]
 [3.69907267]
 [4.18108578]
 [2.82001407]
 [1.69218545]
 [3.24487452]
 [4.09119928]
 [3.49742408]
 [3.80219419]
 [2.36875543]
 [2.92640614]
 [1.83354524]
 [3.14938959]
 [3.25696779]
 [2.54375308]
 [2.22778599]
 [3.38903852]]
测试集Y预测值： [3.7399747 3.0461605 3.6443188 3.1921263 3.4937832 3.2169008 2.8351967
 3.7082036 3.911745  3.2307427 2.0557182 3.5011365 4.141977  3.507895
 3.3324695 2.6794558 2.7649722 2.2146606 2.9744594 2.8480847 2.7587197
 2.5051935 3.4968247]
训练集Y： [[3.68459797]
 [2.5437363 ]
 [3.8552464 ]
 [3.74136833]
 [3.89323245]
 [4.10219311]
 [3.2851029 ]
 [3.7411158 ]
 [2.62564884]
 [3.61694543]
 [1.70888789]
 [3.26544019]
 [5.77725612]
 [4.18560034]
 [2.11004125]
 [1.59137159]
 [3.51160147]
 [4.40567336]
 [0.89243119]
 [3.115013

In [43]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Your original model setup
xgb_model = XGBRegressor(random_state=42,
                         learning_rate=0.188,
                         max_depth=6,
                         n_estimators=42,
                         reg_alpha=0.001,  # L1 regularization
                         reg_lambda=1)

# Define a K-Fold cross-validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')

# Calculate RMSE for each fold
rmse_scores = np.sqrt(-cv_scores)

print(f'RMSE scores for each fold: {rmse_scores}')
print(f'Mean RMSE: {rmse_scores.mean()}')


RMSE scores for each fold: [1.08918157 0.90928255 0.92612992 0.75474935 0.86086645]
Mean RMSE: 0.908041965961855
